# Creación del Scraper

## Importamos nuestras librerías 

In [82]:
import pandas as pd
import time 
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

## Creamos la clase `googleMapsScraper` 

Contendrá las funciones principales para abrir el navegador, entrar a Google Maps, ejecutar una búsqueda, bajar el scroll hasta que dejen de salir resultados y devolver los nombres y URLs recopilados de los lugares.

In [86]:
class googleMapsScraper:
#################################################################
    def __init__(self): 
        #Inicializar el Driver
        driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()))
        self.driver = driver
#################################################################

    def searchPlaces(self, stringSearch):
        # Lista de resultados de la busqueda
        self.results = []

        try:
            searchText = self.driver.find_element(By.CLASS_NAME, "tactile-searchbox-input")
            searchText.send_keys(stringSearch)
            searchButton = self.driver.find_element(By.XPATH, '//html//body//div[3]//div[9]//div[3]//div[1]//div[1]//div[1]//div[2]//div[1]//button')
            searchButton.click()

            # Se tiene que generar antes de poder interactuar, por ello le damos 5 segundos
            time.sleep(5)

            # Ubicando elementos de la página 
            resultsBox= self.driver.find_element(By.XPATH, "/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]")

            # Acciones dentro del navegador
            action = ActionChains(self.driver)

            if (resultsBox.is_displayed() == True): # Si se están mostrando los resultados
                action.move_to_element(resultsBox) # Moverse a la sección de resultados


                initial_len = 0
                while(resultsBox.is_displayed() == True):
                    page_content = self.driver.page_source # Obteniendo contenido de la pagina inicial
                    response = Selector(page_content) 
                    initial_len = (len(response.xpath('//div[contains(@aria-label, "Resultados de")]/div/div[./a]'))) 

                    for i in range(5): # Para darle tiempo de actualizar busquedas y evitar que se traslapen
                        time.sleep(1) 
                        resultsBox.send_keys(Keys.PAGE_DOWN) # Scroll hacia abajo
                    
                    page_content = self.driver.page_source # Obteniendo contenido de la pagina después de actualizar
                    response = Selector(page_content) 
                    actual_len = (len(response.xpath('//div[contains(@aria-label, "Resultados de")]/div/div[./a]'))) # Obtenemos la cantidad de busquedas

                    if(initial_len == actual_len and actual_len!= 0):
                        break

                for el in response.xpath('//div[contains(@aria-label, "Resultados de")]/div/div[./a]'):
                    self.results.append({
                        'Nombre del lugar': el.xpath('./a/@aria-label').extract_first(''),
                        'Sitio en Google Maps': el.xpath('./a/@href').extract_first('')
                })
            return(self.results)
            
        except:
            pass
#################################################################
    def run(self, stringSearch):
        self.driver.get('https://www.google.com/maps')
        time.sleep(2)
        data = self.searchPlaces(stringSearch)
        return(data)

## Asignamos la URL de Maps y el input de la búsqueda

In [87]:

stringSearch = " \"CDMX, GAM\"" + " " + "+" + " " + "\"Cafeteria\" "

## Ejecutamos la clase anterior y guardamos los datos

In [88]:
mapsScraper = googleMapsScraper()
data = mapsScraper.run(url, stringSearch)

## Guardamos la información en un `DataFrame`

In [89]:
names = pd.DataFrame(data)
names

,Nombre del lugar,Sitio en Google Maps
0,Cafeteria LUCY,https://www.google.com/maps/place/Cafeteria+LU...
1,Café el grinch,https://www.google.com/maps/place/Caf%C3%A9+el...
2,CafeTería 4,https://www.google.com/maps/place/CafeTer%C3%A...
3,Cafeteria Kawet,https://www.google.com/maps/place/Cafeteria+Ka...
4,Más Que Café,https://www.google.com/maps/place/M%C3%A1s+Que...
...,...,...
58,El Cafe y Compañia,https://www.google.com/maps/place/El+Cafe+y+Co...
59,Cafe & Internet,https://www.google.com/maps/place/Cafe+%26+Int...
60,Café La Puerta,https://www.google.com/maps/place/Caf%C3%A9+La...
61,Cafeteria,https://www.google.com/maps/place/Cafeteria/da...


---

## Creamos la función `completeDataCollect()`

Está función es la más tardada y más importante, ya que se encarga de entrar a cada una de las URLs que obtuvimos anteriormente y recopilar toda la información importante del sitio en cuestión. Entre los datos recopilados tenemos la ubicación, cantidad de estrellas, conteo de reviews, sitio web, número de teléfono y horarios.

In [98]:
def completeDataCollect(url):

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=options)


    driver.get(url)


    page_content = driver.page_source # Obteniendo contenido de la 
    response = Selector(page_content)
    try:
        location = response.xpath('//button[contains(@aria-label, "Dirección:")]').xpath("@aria-label").extract_first().replace("Dirección:","")
    except:
        location = "N/A"

    try:  
        stars = response.xpath('//span[contains(@aria-label, "estrellas")]').xpath("@aria-label").extract_first().split()[0]
    except:
        stars = "N/A"

    try:
        reviewsCount = response.xpath('//button[contains(@aria-label, "opiniones")]').xpath("@aria-label").extract_first().split()[0]
    except:
        reviewsCount = "N/A"

    try:
        webSite = response.xpath('//a[contains(@aria-label, "Sitio web:")]').xpath("@href").extract_first()
    except:
        webSite = "N/A"

    try:
        phoneNumber = response.xpath('//button[contains(@aria-label, "Teléfono:")]').xpath("@aria-label").extract_first().replace("Teléfono: ","")
    except:
        phoneNumber = "N/A"

    try:
        sched = response.xpath('//div[contains(@aria-label, "horario de apertura durante la semana")]').xpath("@aria-label").extract_first("")
    except:
        sched = "N/A"
    
    return (location,stars,reviewsCount,webSite,phoneNumber,sched)

## Guardamos los datos de cada ubicación en una lista

In [95]:
results = []

for i in range(len(names)):
    results.append(completeDataCollect(names["Sitio en Google Maps"][i]))

## Generamos una lista para cada columna de información

In [96]:
addres_ = []
stars_ = []
reviewsCount_ = []
webSite_ = []
phone_ = []
sched_ = []

for i in range(len(names)):
    addres_.append(results[i][0])
    stars_.append(results[i][1])
    reviewsCount_.append(results[i][2])
    webSite_.append(results[i][3])
    phone_.append(results[i][4])
    sched_.append(results[i][5])

## Las asignamos a un diccionario con sus llaves correspondientes

In [97]:
dict_ = {"Dirección": addres_,
        "Calificación":stars_,
        "Total de Calificaciones": reviewsCount_,
        "Stitio web": webSite_,
        "Teléfono": phone_,
        "Horario": sched_
}

df_ = pd.DataFrame(dict_) 
df_  

,Dirección,Calificación,Total de Calificaciones,Stitio web,Teléfono,Horario
0,"Tres Anegas 88, Ampliación Progreso Nacional,...",4.5,2,https://www.facebook.com/LUCY-LSD-436353070141...,55 2450 8830,55 2450 8830
1,"Primera Cda. de Apango 3, Gral Felipe Berrioz...",4.8,12,None,N/A,N/A
2,"Av. Juan de Dios Bátiz, Nueva Industrial Vall...",5.0,N/A,None,N/A,N/A
3,"Cerca de, Cda. Mauricio Gómez 40b, Gral Felip...",3.6,13,None,55 8364 9692,55 8364 9692
4,"Cda. Juventino Rosas 250, Juventino Rosas, Gu...",4.3,60,https://masquecafedeldirecto.negocio.site/,55 6325 5219,55 6325 5219
...,...,...,...,...,...,...
58,"Av. Montevideo 363, Lindavista Sur, Gustavo A...",4.3,7,None,55 1421 5966,55 1421 5966
59,"Calz. Ticomán, La Laguna Ticoman, Gustavo A. ...",5.0,2,None,N/A,N/A
60,"Industrial, Gustavo A. Madero, 07800 Ciudad d...",4.3,3,None,N/A,N/A
61,"Heraldo de Toluca, Prensa Nacional, 54170 Tla...",5.0,Buscar,None,N/A,N/A


## Organizamos la información y terminamos el `DataFrame` final

In [96]:
data_df = pd.concat([names, df_], axis=1)
data_df

,Nombre del lugar,Sitio en Google Maps,Dirección,Calificación,Total de Calificaciones,Stitio web,Teléfono,Horario
0,Cafeteria LUCY,https://www.google.com/maps/place/Cafeteria+LU...,"Tres Anegas 88, Ampliación Progreso Nacional,...",4.5,2,https://www.facebook.com/LUCY-LSD-436353070141...,55 2450 8830,55 2450 8830
1,Café el grinch,https://www.google.com/maps/place/Caf%C3%A9+el...,"Primera Cda. de Apango 3, Gral Felipe Berrioz...",4.9,10,None,N/A,N/A
2,CafeTería 4,https://www.google.com/maps/place/CafeTer%C3%A...,"Av. Juan de Dios Bátiz, Nueva Industrial Vall...",5.0,N/A,None,N/A,N/A
3,CAFE 1964,https://www.google.com/maps/place/CAFE+1964/da...,"Calle 13 esquina, C. Monte Alto Col, Progreso...",4.6,132,None,55 5392 9179,55 5392 9179
4,Cafetería,https://www.google.com/maps/place/Cafeter%C3%A...,"Ote. 157 3802, Salvador Díaz Mirón, Gustavo A...",4.4,14,None,N/A,N/A
...,...,...,...,...,...,...,...,...
59,Cafe & Internet,https://www.google.com/maps/place/Cafe+%26+Int...,"Calz. Ticomán, La Laguna Ticoman, Gustavo A. ...",5.0,2,None,N/A,N/A
60,Café Acapulco,https://www.google.com/maps/place/Caf%C3%A9+Ac...,"C. 28 199, Guadalupe Proletaria, Gustavo A. M...",5.0,Buscar,None,N/A,N/A
61,Café La Puerta,https://www.google.com/maps/place/Caf%C3%A9+La...,"Industrial, Gustavo A. Madero, 07800 Ciudad d...",4.3,3,None,N/A,N/A
62,Cafeteria,https://www.google.com/maps/place/Cafeteria/da...,"Heraldo de Toluca, Prensa Nacional, 54170 Tla...",5.0,Buscar,None,N/A,N/A


---